In [1]:
import gensim
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [57]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-600.model')
model.init_sims(replace=True)

2017-05-03 11:30:02,284 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-05-03 11:30:09,433 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.* with mmap=None
2017-05-03 11:30:09,433 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.syn0.npy with mmap=None
2017-05-03 11:30:17,298 : INFO : setting ignored attribute syn0norm to None
2017-05-03 11:30:17,326 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.syn1neg.npy with mmap=None
2017-05-03 11:30:56,800 : INFO : setting ignored attribute cum_table to None
2017-05-03 11:30:56,911 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-05-03 11:32:27,656 : INFO : precomputing L2-norms of word weight vectors


In [14]:
num_features = 600

In [6]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [7]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [12]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

In [10]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [11]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [64]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [65]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-05-03 11:37:10,349 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:11,083 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:11,092 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:12,166 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:12,546 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:13,217 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:14,312 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:14,435 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:14,775 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:15,007 : WARNING : cannot compute similarity with no input []
2017-05-03 11:37:15,083 : WARNING : cannot compute similarity with no input []


CPU times: user 5.88 s, sys: 327 ms, total: 6.21 s
Wall time: 8.95 s


In [66]:
del model

### LIWC Attr

In [27]:
personalities = pd.read_csv('personality-normalized.csv')

In [28]:
personalities = personalities.convert_objects(convert_numeric=True)

/usr/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [29]:
personalities.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [30]:
train_data, test_data = train_test_split(personalities, test_size=0.3, random_state=42)

In [31]:
liwc_attr = ['X1funct' ,'X2pronoun' ,'X3ppron','X4i','X5we','X6you','X7shehe','X8they','X9ipron','X10article','X11verb','X12auxverb','X13past','X14present','X15future','X16adverb','X17preps','X18conj','X19negate','X20quant','X21number','X22swear','X23social','X24family','X25friend','X26humans','X27affect','X28posemo','X29negemo','X30anx','X31anger','X32sad','X33cogmech','X34insight','X35cause','X36discrep','X37tentat','X38certain','X39inhib','X40incl','X41excl','X42percept','X43see','X44hear','X45feel','X46bio','X47body','X48health','X49sexual','X50ingest','X51relativ','X52motion','X53space','X54time','X55work','X56achieve','X57leisure','X58home','X59money','X60relig','X61death','X62assent','X63nonfl', 'X64filler']

In [32]:
def remove_nan(atributes):
    train_attributes = np.array(train_data[atributes])
    test_attributes = np.array(test_data[atributes])
    train_attributes[np.isnan(train_attributes)] = 0
    test_attributes[np.isnan(test_attributes)] = 0
    return (train_attributes, test_attributes)

In [33]:
train, test = remove_nan(liwc_attr)

## Dictionairies + word embeddings

In [74]:
print train.shape
print X_train_word_average.shape

(727, 64)
(727, 600)


In [25]:
def merge_attributes(conj1, conj2):
    combined_attr = []

    for i in range(0,conj1.shape[0]):
        mixed_attr = np.concatenate((conj1[i], conj2[i]))
        combined_attr.append(mixed_attr)
    return np.vstack(combined_attr)

In [76]:
train_merged = merge_attributes(train, X_train_word_average)
test_merged = merge_attributes(test, X_test_word_average)

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [78]:
%%time
model_trainer.fit(train_merged, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.8s finished


CPU times: user 5.49 s, sys: 495 ms, total: 5.99 s
Wall time: 35.3 s


In [79]:
yp = model.predict(test_merged)
yt = test_w2v_data['extraversion_m']

In [80]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,106,36
1,82,88


In [81]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.56      0.75      0.64       142
          1       0.71      0.52      0.60       170

avg / total       0.64      0.62      0.62       312



In [82]:
accuracy_score(yt, yp)

0.62179487179487181

## Remove stop words

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-600.model')
model.init_sims(replace=True)

2017-05-03 12:14:17,627 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-05-03 12:14:24,035 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.* with mmap=None
2017-05-03 12:14:24,035 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.syn0.npy with mmap=None
2017-05-03 12:14:31,866 : INFO : setting ignored attribute syn0norm to None
2017-05-03 12:14:31,867 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.syn1neg.npy with mmap=None
2017-05-03 12:15:07,127 : INFO : setting ignored attribute cum_table to None
2017-05-03 12:15:07,337 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-05-03 12:16:36,943 : INFO : precomputing L2-norms of word weight vectors


In [3]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            if word in stopwords.words('portuguese'):
                continue
            tokens.append(word)
    return tokens

In [8]:
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [15]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-05-03 12:28:31,412 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:31,971 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:31,979 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:32,803 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:33,098 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:33,551 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:34,386 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:34,482 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:34,728 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:34,915 : WARNING : cannot compute similarity with no input []
2017-05-03 12:28:34,973 : WARNING : cannot compute similarity with no input []


CPU times: user 3.82 s, sys: 78.4 ms, total: 3.9 s
Wall time: 4.6 s


In [16]:
del model

In [19]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.1s finished


CPU times: user 5.62 s, sys: 340 ms, total: 5.96 s
Wall time: 34.3 s


In [20]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['extraversion_m']

In [21]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,100,42
1,82,88


In [22]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.55      0.70      0.62       142
          1       0.68      0.52      0.59       170

avg / total       0.62      0.60      0.60       312



In [23]:
accuracy_score(yt, yp)

0.60256410256410253

## W2V Stop + LIWC

In [34]:
train_merged = merge_attributes(train, X_train_word_average)
test_merged = merge_attributes(test, X_test_word_average)

In [35]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [36]:
%%time
model_trainer.fit(train_merged, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   27.2s finished


CPU times: user 5.4 s, sys: 87.9 ms, total: 5.49 s
Wall time: 32.7 s


In [37]:
yp = model.predict(test_merged)
yt = test_w2v_data['extraversion_m']

In [38]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,104,38
1,84,86


In [39]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.55      0.73      0.63       142
          1       0.69      0.51      0.59       170

avg / total       0.63      0.61      0.61       312



In [40]:
accuracy_score(yt, yp)

0.60897435897435892